In [1]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

### Import libraries 

In [2]:
import numpy as np                    # numerical cumputations
import pandas as pd                   # data processing
import re                             # Regular expression
import os                             # operating system


from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer   # Modelling


import matplotlib.pyplot as plt
%matplotlib inline


from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 8,6

pd.set_option('max_colwidth', 6000)

### Data collection 

#### removing observations without data 

In [3]:
data = pd.read_csv('input.csv', encoding='cp437')
data

,label,text
0,Banking,"Account Manager - Pensions Salary: circa. ┬ú31,000 + competitive commission scheme + benefits (28 days' holiday + annual bonus + pension + flex benefits) Work for the largest mutual life, pensions and investment company in the UK, with Group funds under management of ┬ú100 billion. Group businesses provide around 9.1 million policies and employ 3,253 people. (Figures quoted are as at 31 December 2016). We have an exciting Permanent opportunity for an Account Manager to join the Intermediary Account Management Team in Edinburgh in a high performing, hybrid panel role. Working from a central Edinburgh based office, the purpose of this role is to deliver agreed new business targets from an existing panel of Intermediaries through providing a high level of service , maintaining excellent working relationships with key clients and promoting the provider's proposition. Responsibilities: Deliver personal sales targets from a panel of Intermediaries Develop new Intermediary relationships Deliver sales presentations to Intermediaries promoting new and existing sales opportunities Identify and promote new sales opportunities by telephone & face to face Develop and maintain strong business relationships with panel Intermediaries Ensure a smooth efficient and professional sales process for the customer Work with Intermediaries to ensure an excellent and professional service is provided Engage, align and work with appropriate colleagues and resources to maximise profitable new business production and to protect existing business Communicate requirements clearly and effectively with colleagues to ensure customer needs are identified and satisfied Create, maintain and deliver plan to develop a panel of intermediaries Skills, Qualifications & Experience: Experience within a similar sales or account management role within Pensions Knowledge and understanding of the IFA / Intermediary market RO1 & RO4 Qualifications would be advantageous Confident in understanding technology used by advisers including research, analysis systems, platforms etc Experience in dealing with intermediaries in a face to face or telephone role Excellent communication skills both written and verbal Ability to discuss and debate industry issues, technical & legislative updates in a competent and influential manner To apply, please use the 'Apply Online' link below. For any further queries regarding the role, please contact ************************************** or call ************ Apply Now 1-Click Apply Ref no: 105707-AMUPEN_2657339941 This vacancy is being advertised by Sanderson Recruitment. The services advertised by Sanderson Recruitment are those of an Employment Agency."
1,Banking,"Account Manager - Pensions Salary: circa. ┬ú31,000 + competitive commission scheme + benefits (28 days' holiday + annual bonus + pension + flex benefits) Work for the largest mutual life, pensions and investment company in the UK, with Group funds under management of ┬ú100 billion. Group businesses provide around 9.1 million policies and employ 3,253 people. (Figures quoted are as at 31 December 2016). We have an exciting Permanent opportunity for an Account Manager to join the Intermediary Account Management Team in Edinburgh in a high performing, hybrid panel role. Working from a central Edinburgh based office, the purpose of this role is to deliver agreed new business targets from an existing panel of Intermediaries through providing a high level of service , maintaining excellent working relationships with key clients and promoting the provider's proposition. Responsibilities: * Deliver personal sales targets from a panel of Intermediaries * Develop new Intermediary relationships * Deliver sales presentations to Intermediaries promoting new and existing sales opportunities * Identify and promote new sales opportunities by telephone & face to face * Develop and maintain strong business relationships with panel Intermediaries * Ensure a smooth efficient and pro

## Data Preprocessing 

####  Clean data using regular expression

In [5]:
data['text'] = data['text'].apply(lambda x: re.sub(r"[|‘|’|'|>|<|_|¢|#|£|+|&|]",r' ',x))
data['text'] = data['text'].apply(lambda x: re.sub(r'[|^|=|~|…|,|*|”|“|"|"|%|/|@|(|)|]',r' ',x))
data['text'] = data['text'].apply(lambda x : re.sub("[\(\[].*?[\)\]]", r' ', x))
data['text'] = data['text'].apply(lambda x: re.sub(r'[?|$|.|!|]',r' ',x))
data['text'] = data.text.str.replace("-"," ")
data['text'] = data.text.str.replace("—"," ")
data['text'] = data.text.str.replace("–"," ")
data['text'] = data.text.str.replace(":"," ")
data['text'] = data.text.str.replace("¦"," ")
data['text'] = data.text.str.replace(";"," ")
data['text'] = data.text.str.replace("â€™"," ")
data['text'] = data.text.str.replace("â€"," ")
data['text'] = data.text.str.replace("â","a")
data['text'] = data.text.str.replace("Â","a")
data['text'] = data.text.str.replace("┬ú"," ")

data.head(1)

,label,text
0,Banking,Account Manager Pensions Salary circa 31 000 competitive commission scheme benefits 28 days holiday annual bonus pension flex benefits Work for the largest mutual life pensions and investment company in the UK with Group funds under management of 100 billion Group businesses provide around 9 1 million policies and employ 3 253 people Figures quoted are as at 31 December 2016 We have an exciting Permanent opportunity for an Account Manager to join the Intermediary Account Management Team in Edinburgh in a high performing hybrid panel role Working from a central Edinburgh based office the purpose of this role is to deliver agreed new business targets from an existing panel of Intermediaries through providing a high level of service maintaining excellent working relationships with key clients and promoting the provider s proposition Responsibilities Deliver personal sales targets from a panel of Intermediaries Develop new Intermediary relationships Deliver sales presentations to Intermediaries promoting new and existing sales opportunities Identify and promote new sales opportunities by telephone face to face Develop and maintain strong business relationships with panel Intermediaries Ensure a smooth efficient and professional sales process for the customer Work with Intermediaries to ensure an excellent and professional service is provided Engage align and work with appropriate colleagues and resources to maximise profitable new business production and to protect existing business Communicate requirements clearly and effectively with colleagues to ensure customer needs are identified and satisfied Create maintain and deliver plan to develop a panel of intermediaries Skills Qualifications Experience Experience within a similar sales or account management role within Pensions Knowledge and understanding of the IFA Intermediary market RO1 RO4 Qualifications would be advantageous Confident in understanding technology used by advisers including research analysis systems platforms etc Experience in dealing with intermediaries in a face to face or telephone role Excellent communication skills both written and verbal Ability to discuss and debate industry issues technical legislative updates in a competent and influential manner To apply please use the Apply Online link below For any further queries regarding the role please contact or call Apply Now 1 Click Apply Ref no 105707 AMUPEN 2657339941 This vacancy is being advertised by Sanderson Recruitment The services advertised by Sanderson Recruitment are those of an Employment Agency


#### Remove Lower case words

In [6]:
data['text'] = data['text'].apply(lambda x : x.lower())

#### Remove stopwords

In [7]:
# Update stopword list - Customization
eng_stopwords.extend(['now','click','apply','ref','mv' , 'etc'])

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (eng_stopwords)]))

In [32]:
# Juck words starting with y

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('γ')]))

#### Remove digits 

In [33]:
data['text'] = data['text'].apply(lambda x: re.sub(r"\d+",r'',x))

#### Remove extra spaces

In [34]:
data['text'] = data['text'].apply(lambda x: re.sub("\s\s+", " ", x))

data.head(1)

,label,text
0,Banking,account manager pension salary circa competitive commission scheme benefit day holiday annual bonus pension flex benefit work largest mutual life pension investment company group fund management billion group business provide around million policy employ people figure quoted december exciting permanent opportunity account manager join intermediary account management team edinburgh high performing hybrid panel role working central edinburgh based office purpose role deliver agreed new business target existing panel intermediary providing high level service maintaining excellent working relationship key client promoting provider proposition responsibility deliver personal sale target panel intermediary develop new intermediary relationship deliver sale presentation intermediary promoting new existing sale opportunity identify promote new sale opportunity telephone face face develop maintain strong business relationship panel intermediary ensure smooth efficient professional sale process customer work intermediary ensure excellent professional service provided engage align work appropriate colleague resource maximise profitable new business production protect existing business communicate requirement clearly effectively colleague ensure customer need identified satisfied create maintain deliver plan develop panel intermediary skill qualification experience experience within similar sale account management role within pension knowledge understanding ifa intermediary market qualification would advantageous confident understanding technology used adviser including research analysis system platform experience dealing intermediary face face telephone role excellent communication skill written verbal ability discus debate industry issue technical legislative update competent influential manner please use online link query regarding role please contact call amupen vacancy advertised sanderson recruitment service advertised sanderson recruitment employment agency


### Lematiation

In [27]:
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(word) for word in x.split() ]))
data.head(1)

,label,text
0,Banking,account manager pension salary circa competitive commission scheme benefit day holiday annual bonus pension flex benefit work largest mutual life pension investment company group fund management billion group business provide around million policy employ people figure quoted december exciting permanent opportunity account manager join intermediary account management team edinburgh high performing hybrid panel role working central edinburgh based office purpose role deliver agreed new business target existing panel intermediary providing high level service maintaining excellent working relationship key client promoting provider proposition responsibility deliver personal sale target panel intermediary develop new intermediary relationship deliver sale presentation intermediary promoting new existing sale opportunity identify promote new sale opportunity telephone face face develop maintain strong business relationship panel intermediary ensure smooth efficient professional sale process customer work intermediary ensure excellent professional service provided engage align work appropriate colleague resource maximise profitable new business production protect existing business communicate requirement clearly effectively colleague ensure customer need identified satisfied create maintain deliver plan develop panel intermediary skill qualification experience experience within similar sale account management role within pension knowledge understanding ifa intermediary market qualification would advantageous confident understanding technology used adviser including research analysis system platform experience dealing intermediary face face telephone role excellent communication skill written verbal ability discus debate industry issue technical legislative update competent influential manner please use online link query regarding role please contact call amupen vacancy advertised sanderson recruitment service advertised sanderson recruitment employment agency


#### Remove words with less than 3 characters 

In [35]:
data['text'] = data['text'].apply(lambda x: re.sub(r'\W*\b\w{1,2}\b',r' ',x))
data.head(1)

,label,text
0,Banking,account manager pension salary circa competitive commission scheme benefit day holiday annual bonus pension flex benefit work largest mutual life pension investment company group fund management billion group business provide around million policy employ people figure quoted december exciting permanent opportunity account manager join intermediary account management team edinburgh high performing hybrid panel role working central edinburgh based office purpose role deliver agreed new business target existing panel intermediary providing high level service maintaining excellent working relationship key client promoting provider proposition responsibility deliver personal sale target panel intermediary develop new intermediary relationship deliver sale presentation intermediary promoting new existing sale opportunity identify promote new sale opportunity telephone face face develop maintain strong business relationship panel intermediary ensure smooth efficient professional sale process customer work intermediary ensure excellent professional service provided engage align work appropriate colleague resource maximise profitable new business production protect existing business communicate requirement clearly effectively colleague ensure customer need identified satisfied create maintain deliver plan develop panel intermediary skill qualification experience experience within similar sale account management role within pension knowledge understanding ifa intermediary market qualification would advantageous confident understanding technology used adviser including research analysis system platform experience dealing intermediary face face telephone role excellent communication skill written verbal ability discus debate industry issue technical legislative update competent influential manner please use online link query regarding role please contact call amupen vacancy advertised sanderson recruitment service advertised sanderson recruitment employment agency


## Modelling 

#### Converting dataset into tf-idf 

In [70]:
n_features = 4000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(data.text)

In [46]:
import pickle
pickle.dump(tfidf_vectorizer.vocabulary_,open("vocab.pkl","wb"))

#### Converting into Dataframe with featurename

In [71]:
tfidf = pd.DataFrame(tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
tfidf.head()

,abbot,ability,able,aboutyourecruit,abpi,abreast,abroad,absence,aca,academia,...,www,wycombe,xamarin,xaml,xml,xpath,xslt,zend,zone,óeur
0,0.0,0.019733,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.019947,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.089148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.089027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.041985,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Converting into numpy array , which later to be consumed by Model 

In [72]:
data_features = tfidf.values

In [73]:
dict_tgt = {'Banking':1,
 'Construction':2,
 'Contruction':2,
 'Energy':3,
 'Health':4,
 'Insurance':5,
 'Manufacturing':6,
 'Oil and Gas':7,
 'Technology':8}

train_labels = data.label.map(dict_tgt)

#### Splitting dataset into train and test 

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_features, data_lables, test_size=0.30, random_state=492)
print(X_train.shape)
print(X_test.shape)

(3189, 4000)
(1368, 4000)


### Naive Bayes Algorithm

In [75]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train, y_train)

print('Training Score',gnb.score(X_train, y_train))
print('testing Score',gnb.score(X_test, y_test))

Training Score 0.949827532142
testing Score 0.90716374269


### Logistic Regression Algorithm

In [76]:
# so we will use aplha as 10 because post that test error no more decrease thus model overfits afterwards 

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)  
lr.fit(X_train, y_train)

print('Training Score',lr.score(X_train, y_train))
print('testing Score',lr.score(X_test, y_test))
print("Number of features used: {}".format(np.sum(lr.coef_ != 0)))

Training Score 0.960489181562
testing Score 0.948830409357
Number of features used: 66


### Random Forest

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search

rf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
parameters = {'max_features':['sqrt', 'log2'],
              'max_depth':[5,10]}

clf_grid = grid_search.GridSearchCV(rf, parameters)
clf_grid.fit(X_train, y_train)

print('Training Score',clf_grid.score(X_train, y_train))
print('testing Score',clf_grid.score(X_test, y_test))

Training Score 0.97177798683
testing Score 0.937134502924
